ЗДЕСЬ МЫ РАЗРАБАТЫВЕМ ЛИНЕЙНУЮ МОДЕЛЬ ДЛЯ БОЛЕЕ ТОЧНОГО РАССЧЕТА СКОРОВ
Предлагается взять данные из статьи 2018, расширить пространство параметров, которые подаются на вход модели при обучении и выбрать часть данных, которая нам нужна (нужные файлы)


In [ ]:
!pip install biopython

In [ ]:
!wget http://rna.urmc.rochester.edu/Releases/current/RNAstructureLinuxTextInterfaces64bit.tgz

In [ ]:
!wget 'https://static-content.springer.com/esm/art%3A10.1038%2Fnbt.4061/MediaObjects/41587_2018_BFnbt4061_MOESM39_ESM.xlsx' -O table1.xlsx

In [ ]:
!git clone https://github.com/MyungjaeSong/Paired-Library.git

In [7]:
import pandas as pd
from Bio.Seq import Seq

Экспрот данных их статьи https://doi.org/10.1038/nbt.4061


In [8]:
data = pd.read_excel('table1.xlsx', header=1).iloc[:-2]
data.columns = [
    '50bp_seq',
    '34bp_seq',
    'guide_seq',
    'indel_freq_background',
    'indel_read_count_background',
    'total_read_count_background',
    'indel_freq_delivered',
    'indel_read_count_delivered',
    'total_read_count_delivered',
    'indel_freq_background_subtracted'
]
data

,50bp_seq,34bp_seq,guide_seq,indel_freq_background,indel_read_count_background,total_read_count_background,indel_freq_delivered,indel_read_count_delivered,total_read_count_delivered,indel_freq_background_subtracted
0,TGCGCGAGCGTTTAAAAAACATCGAACGCATCTGCTGCCTAGCTTG...,AGCGTTTAAAAAACATCGAACGCATCTGCTGCCT,AAAAACATCGAACGCATCTG,0.191205,1.0,523.0,14.902507,107.0,718.0,14.711302
1,CTAAAGAAACTTTAAAAATCTTTTCTGCCAGATCTCCAGAAGCTTG...,AAACTTTAAAAATCTTTTCTGCCAGATCTCCAGA,AAAATCTTTTCTGCCAGATC,0.000000,0.0,486.0,0.238095,1.0,420.0,0.238095
2,TTGCCATTGTTTTAAAACAGGTTCTGTACTTGATCTCTCCAGCTTG...,TTGTTTTAAAACAGGTTCTGTACTTGATCTCTCC,AAACAGGTTCTGTACTTGAT,0.178571,1.0,560.0,88.258317,451.0,511.0,88.079746
3,TTGCACATATTTTAAAACTGAGTTCAAAGACCACTCTTCCAGCTTG...,ATATTTTAAAACTGAGTTCAAAGACCACTCTTCC,AAACTGAGTTCAAAGACCAC,0.000000,0.0,301.0,75.392670,288.0,382.0,75.392670
4,TAGACTAATGTTTAAAAGCAAGTGCAAGTCTTTGGAATCTAGCTTG...,AATGTTTAAAAGCAAGTGCAAGTCTTTGGAATCT,AAAGCAAGTGCAAGTCTTTG,0.000000,0.0,460.0,63.320080,637.0,1006.0,63.320080
...,...,...,...,...,...,...,...,...,...,...
14995,TCCATCTTCATTTTTTTTGTAGAGTAGGGCTTTATTTCCAAGCTTG...,TTCATTTTTTTTGTAGAGTAGGGCTTTATTTCCA,TTTTGTAGAGTAGGGCTTTA,0.467290,1.0,214.0,0.000000,0.0,209.0,-0.467290
14996,CCTTCTCTCCTTTTTTTTTCAAGATCTGATTCTTCTTGCAAGCTTG...,CTCCTTTTTTTTTCAAGATCTGATTCTTCTTGCA,TTTTTCAAGATCTGATTCTT,0.000000,0.0,388.0,0.000000,0.0,293.0,0.000000
14997,CCAGGACTTGTTTTTTTTTCAATCTGTTCATCTTGGACCAAGCTTG...,CTTGTTTTTTTTTCAATCTGTTCATCTTGGACCA,TTTTTCAATCTGTTCATCTT,0.000000,0.0,2125.0,0.239006,5.0,2092.0,0.239006
14998,ACCATCATAATTTTTTTTTGCAACATAGCCATTTCTTTTTAGCTTG...,ATAATTTTTTTTTGCAACATAGCCATTTCTTTTT,TTTTTGCAACATAGCCATTT,0.357143,1.0,280.0,0.084317,1.0,1186.0,-0.272826


Расширение параметров - анализ ГЦ-состава, полиандениновых районов, анализ вторичной структуры

In [9]:
# Set GC content and AAA content
data['rna']=0
data['GC_content']=0
data['AAA_content']=0
for index, row in data.iterrows():
  rna=Seq(str(row['guide_seq'])).complement().transcribe() 
  data['GC_content'].iloc[index]=(rna.count("G")+rna.count("C"))/len(rna)
  data['AAA_content'].iloc[index]=(rna.count("AAA"))
  data['rna'].iloc[index]=str(rna)
  

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
!wget https://www.tbi.univie.ac.at/RNA/download/sourcecode/2_4_x/ViennaRNA-2.4.17.tar.gz
!tar -zxvf ViennaRNA-2.4.17.tar.gz
!cd ViennaRNA-2.4.17 ; ./configure; make ; sudo make install
import sys
sys.path.append("/content/ViennaRNA-2.4.17/interfaces/Python3/RNA")
import _RNA as RNA
import csv


In [12]:
# Measure secondary strucute elements content
data['sec_str']=0
for index, row in data.iterrows():
  (ss, mfe) = RNA.fold(str(row['rna']))
  data['sec_str'].iloc[index]=ss.count('.')


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [13]:
data


,50bp_seq,34bp_seq,guide_seq,indel_freq_background,indel_read_count_background,total_read_count_background,indel_freq_delivered,indel_read_count_delivered,total_read_count_delivered,indel_freq_background_subtracted,rna,GC_content,AAA_content,sec_str
0,TGCGCGAGCGTTTAAAAAACATCGAACGCATCTGCTGCCTAGCTTG...,AGCGTTTAAAAAACATCGAACGCATCTGCTGCCT,AAAAACATCGAACGCATCTG,0.191205,1.0,523.0,14.902507,107.0,718.0,14.711302,UUUUUGUAGCUUGCGUAGAC,0.40,0,20
1,CTAAAGAAACTTTAAAAATCTTTTCTGCCAGATCTCCAGAAGCTTG...,AAACTTTAAAAATCTTTTCTGCCAGATCTCCAGA,AAAATCTTTTCTGCCAGATC,0.000000,0.0,486.0,0.238095,1.0,420.0,0.238095,UUUUAGAAAAGACGGUCUAG,0.35,1,12
2,TTGCCATTGTTTTAAAACAGGTTCTGTACTTGATCTCTCCAGCTTG...,TTGTTTTAAAACAGGTTCTGTACTTGATCTCTCC,AAACAGGTTCTGTACTTGAT,0.178571,1.0,560.0,88.258317,451.0,511.0,88.079746,UUUGUCCAAGACAUGAACUA,0.35,0,12
3,TTGCACATATTTTAAAACTGAGTTCAAAGACCACTCTTCCAGCTTG...,ATATTTTAAAACTGAGTTCAAAGACCACTCTTCC,AAACTGAGTTCAAAGACCAC,0.000000,0.0,301.0,75.392670,288.0,382.0,75.392670,UUUGACUCAAGUUUCUGGUG,0.40,0,20
4,TAGACTAATGTTTAAAAGCAAGTGCAAGTCTTTGGAATCTAGCTTG...,AATGTTTAAAAGCAAGTGCAAGTCTTTGGAATCT,AAAGCAAGTGCAAGTCTTTG,0.000000,0.0,460.0,63.320080,637.0,1006.0,63.320080,UUUCGUUCACGUUCAGAAAC,0.40,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,TCCATCTTCATTTTTTTTGTAGAGTAGGGCTTTATTTCCAAGCTTG...,TTCATTTTTTTTGTAGAGTAGGGCTTTATTTCCA,TTTTGTAGAGTAGGGCTTTA,0.467290,1.0,214.0,0.000000,0.0,209.0,-0.467290,AAAACAUCUCAUCCCGAAAU,0.35,2,20
14996,CCTTCTCTCCTTTTTTTTTCAAGATCTGATTCTTCTTGCAAGCTTG...,CTCCTTTTTTTTTCAAGATCTGATTCTTCTTGCA,TTTTTCAAGATCTGATTCTT,0.000000,0.0,388.0,0.000000,0.0,293.0,0.000000,AAAAAGUUCUAGACUAAGAA,0.25,1,12
14997,CCAGGACTTGTTTTTTTTTCAATCTGTTCATCTTGGACCAAGCTTG...,CTTGTTTTTTTTTCAATCTGTTCATCTTGGACCA,TTTTTCAATCTGTTCATCTT,0.000000,0.0,2125.0,0.239006,5.0,2092.0,0.239006,AAAAAGUUAGACAAGUAGAA,0.25,1,20
14998,ACCATCATAATTTTTTTTTGCAACATAGCCATTTCTTTTTAGCTTG...,ATAATTTTTTTTTGCAACATAGCCATTTCTTTTT,TTTTTGCAACATAGCCATTT,0.357143,1.0,280.0,0.084317,1.0,1186.0,-0.272826,AAAAACGUUGUAUCGGUAAA,0.30,2,16


Выберем часть данных, содержащие нужные ПАМы

In [10]:
data['pam'] = data['34bp_seq'].map(lambda x: x[4:9])
pampam = data[data['pam'].str.match('TT[GT]GT')]
pampam

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,50bp_seq,34bp_seq,guide_seq,indel_freq_background,indel_read_count_background,total_read_count_background,indel_freq_delivered,indel_read_count_delivered,total_read_count_delivered,indel_freq_background_subtracted,rna,GC_content,AAA_content,sec_str,pam
10208,CTGGTACCAATTTGTAAACAATATCTTGCATGGTTCTGTCAGCTTG...,CCAATTTGTAAACAATATCTTGCATGGTTCTGTC,TAAACAATATCTTGCATGGT,0.246305,1.0,406.0,63.452915,283.0,446.0,63.206609,AUUUGUUAUAGAACGUACCA,0.30,0,20,TTTGT
10209,AGGCAATCTCTTTGTAAACCGGGTCACTGAAGTCACTCGCAGCTTG...,TCTCTTTGTAAACCGGGTCACTGAAGTCACTCGC,TAAACCGGGTCACTGAAGTC,0.000000,0.0,273.0,26.946108,180.0,668.0,26.946108,AUUUGGCCCAGUGACUUCAG,0.50,0,20,TTTGT
10210,ATTGAAGATATTTGTAACAGTTCTTTGATATTTGGTGATCAGCTTG...,GATATTTGTAACAGTTCTTTGATATTTGGTGATC,TAACAGTTCTTTGATATTTG,0.190840,2.0,1048.0,0.000000,0.0,492.0,-0.190840,AUUGUCAAGAAACUAUAAAC,0.25,2,20,TTTGT
10211,TTCCAGATAATTTGTAACATATGCATAAGAGGCACAGATCAGCTTG...,ATAATTTGTAACATATGCATAAGAGGCACAGATC,TAACATATGCATAAGAGGCA,0.000000,0.0,483.0,27.467811,64.0,233.0,27.467811,AUUGUAUACGUAUUCUCCGU,0.35,0,20,TTTGT
10212,CCAACTGTGATTTGTAACTCCATTTCATGCTTCCTCTCTCAGCTTG...,GTGATTTGTAACTCCATTTCATGCTTCCTCTCTC,TAACTCCATTTCATGCTTCC,0.148810,1.0,672.0,0.000000,0.0,470.0,-0.148810,AUUGAGGUAAAGUACGAAGG,0.40,1,20,TTTGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11360,CTACCGAGGCTTTGTTTTTGGCCGACGTCTCCATGAAGGGAGCTTG...,AGGCTTTGTTTTTGGCCGACGTCTCCATGAAGGG,TTTTTGGCCGACGTCTCCAT,0.274223,3.0,1094.0,0.091324,1.0,1095.0,-0.182899,AAAAACCGGCUGCAGAGGUA,0.50,1,14,TTTGT
11361,GCCTAACTCATTTGTTTTTGGGCTGCGAAGCCATGTAGAGAGCTTG...,CTCATTTGTTTTTGGGCTGCGAAGCCATGTAGAG,TTTTTGGGCTGCGAAGCCAT,0.000000,0.0,202.0,0.000000,0.0,370.0,0.000000,AAAAACCCGACGCUUCGGUA,0.50,1,12,TTTGT
11362,TAAAATGACATTTGTTTTTGTGATTCTGTTGTTTATTTTCAGCTTG...,GACATTTGTTTTTGTGATTCTGTTGTTTATTTTC,TTTTTGTGATTCTGTTGTTT,0.000000,0.0,1839.0,0.000000,0.0,1228.0,0.000000,AAAAACACUAAGACAACAAA,0.25,2,20,TTTGT
11363,CTAAGTCTCATTTGTTTTTTCTTCAAGCTGTGGTTTAACCAGCTTG...,CTCATTTGTTTTTTCTTCAAGCTGTGGTTTAACC,TTTTTTCTTCAAGCTGTGGT,0.000000,0.0,414.0,0.000000,0.0,677.0,0.000000,AAAAAAGAAGUUCGACACCA,0.35,2,20,TTTGT


Вот эту таблицу данных мы используем для обучения и тестирования линейной модели, для уточнения скоров (TODO)

ПОИСК ОФФ-ТАРГЕТОВ 
1. Найдем все ПАМы на примере некоторой хромосомы некоторого организма
2. Необходимо скачать фаста-файл из БД и положить в текущую директорию

In [14]:
import Bio

In [15]:
from Bio import SeqIO

for s_record in SeqIO.parse('sequence.fasta','fasta'):
    name = s_record.id
    seq = s_record.seq
    seqLen = len(s_record)
    print(name,seqLen)
genome=seq 

NC_001133.9 230218


In [16]:
pam1='TTTGT'
pam2='TTGGT'

In [18]:
#from Bio import Align
#aligner = Align.PairwiseAligner()
#aligner.match_score = 1.0
#aligner.mismatch_score = 0
#aligner.gap_score = 0
#alignments1 = aligner.align(pam1, genome)
#alignments2 = aligner.align(pam2, genome)


In [31]:
j=0
for i in range(230213):
  if genome[i:i+5]==pam1:
    j=j+1
print("I found %s PAMs %s in yesst 1 chromosome"%(j, pam1))


I found 499 PAMs TTTGT in yesst 1 chromosome


In [32]:
j=0
for i in range(230213):
  if genome[i:i+5]==pam2:
    j=j+1
print("I found %s PAMs %s in yesst 1 chromosome"%(j, pam1))


I found 419 PAMs TTTGT in yesst 1 chromosome
